In [1]:
# Import dependencies
import pandas as pd
import numpy as np

In [2]:
# Set file path
csvpath = "Resources/2016-precinct-president.csv"

# Read csv and preview dataframe
prec_df = pd.read_csv(csvpath, encoding="ISO-8859-1")
pd.set_option('display.max_columns', None)
prec_df.head()

C:\Users\test\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13,17,19,31,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,year,stage,special,state,state_postal,state_fips,state_icpsr,county_name,county_fips,county_ansi,county_lat,county_long,jurisdiction,precinct,candidate,candidate_normalized,office,district,writein,party,mode,votes,candidate_opensecrets,candidate_wikidata,candidate_party,candidate_last,candidate_first,candidate_middle,candidate_full,candidate_suffix,candidate_nickname,candidate_fec,candidate_fec_name,candidate_google,candidate_govtrack,candidate_icpsr,candidate_maplight
0,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,32.532237,-86.64644,Autauga,10 JONES COMMUNITY CTR,Hillary Clinton,clinton,US President,statewide,False,democratic,election day,135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P00003392,"CLINTON, HILLARY RODHAM / TIMOTHY MICHAEL KAINE",NaN,NaN,NaN,NaN
1,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,32.532237,-86.64644,Autauga,10 JONES COMMUNITY CTR,Gary Johnson,johnson,US President,statewide,False,independent,election day,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P60012234,"JOHNSON, JOHN FITZGERALD MR.",NaN,NaN,NaN,NaN
2,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,32.532237,-86.64644,Autauga,10 JONES COMMUNITY CTR,Jill Stein,stein,US President,statewide,False,independent,election day,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P20003984,"STEIN, JILL",NaN,NaN,NaN,NaN
3,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,32.532237,-86.64644,Autauga,10 JONES COMMUNITY CTR,Donald Trump,trump,US President,statewide,False,republican,election day,218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P80001571,"TRUMP, DONALD J. / MICHAEL R. PENCE",NaN,NaN,NaN,NaN
4,2016,gen,False,Alabama,AL,1,41,Autauga County,1001.0,161526.0,32.532237,-86.64644,Autauga,10 JONES COMMUNITY CTR,[Write-in],in,US President,statewide,True,NaN,election day,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Exploring the data

In [ ]:
# List of Columns
prec_df.columns

In [ ]:
# Getting the value counts for all the states
prec_df['state'].value_counts()

# Minnesota

In [3]:
# Only interested in Minnesota precincts
mn_df = prec_df.loc[prec_df['state'] == 'Minnesota']
mn_df.head()

,year,stage,special,state,state_postal,state_fips,state_icpsr,county_name,county_fips,county_ansi,county_lat,county_long,jurisdiction,precinct,candidate,candidate_normalized,office,district,writein,party,mode,votes,candidate_opensecrets,candidate_wikidata,candidate_party,candidate_last,candidate_first,candidate_middle,candidate_full,candidate_suffix,candidate_nickname,candidate_fec,candidate_fec_name,candidate_google,candidate_govtrack,candidate_icpsr,candidate_maplight
1124002,2016,gen,False,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,Darrell Castle,castle,US President,statewide,False,constitution,total,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P60021102,"CASTLE, DARRELL LANE",NaN,NaN,NaN,NaN
1124003,2016,gen,False,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,Hillary Clinton,clinton,US President,statewide,False,Democratic-Farmer-Labor,total,154,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P00003392,"CLINTON, HILLARY RODHAM / TIMOTHY MICHAEL KAINE",NaN,NaN,NaN,NaN
1124004,2016,gen,False,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,Rocky De La Fuente,fuente,US President,statewide,False,american delta,total,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P60016342,"DE LA FUENTE, ROQUE ROCKY",NaN,NaN,NaN,NaN
1124005,2016,gen,False,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,Gary Johnson,johnson,US President,statewide,False,libertarian,total,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P60012234,"JOHNSON, JOHN FITZGERALD MR.",NaN,NaN,NaN,NaN
1124006,2016,gen,False,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,Jill Stein,stein,US President,statewide,False,green,total,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P20003984,"STEIN, JILL",NaN,NaN,NaN,NaN


In [4]:
# Keep only the columns we're interested in
# Want to come back and remove more once I get a little deeper into it
mn_df = mn_df[['year', 'state', 'state_postal', 'state_fips', 'state_icpsr', 'county_name',
               'county_fips', 'county_ansi', 'county_lat', 'county_long', 'jurisdiction',
               'precinct', 'candidate_normalized', 'mode', 'votes', 'candidate_fec']]
mn_df.head(11)

,year,state,state_postal,state_fips,state_icpsr,county_name,county_fips,county_ansi,county_lat,county_long,jurisdiction,precinct,candidate_normalized,mode,votes,candidate_fec
1124002,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,castle,total,1,P60021102
1124003,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,clinton,total,154,P00003392
1124004,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,fuente,total,0,P60016342
1124005,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,johnson,total,18,P60012234
1124006,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,stein,total,5,P20003984
1124007,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,trump,total,356,P80001571
1124008,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,in,total,4,NaN
1124009,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,mcmullin,total,7,P60022654
1124010,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,kennedy,total,0,NaN
1124011,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,vacek,total,2,NaN


In [5]:
# TEST dataframe to work thru looping thru votes, using two precincts
test_df = mn_df.loc[(mn_df['precinct'] == 'AITKIN TWP|10') | (mn_df['precinct'] == 'AITKIN|5')]
test_df

,year,state,state_postal,state_fips,state_icpsr,county_name,county_fips,county_ansi,county_lat,county_long,jurisdiction,precinct,candidate_normalized,mode,votes,candidate_fec
1124002,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,castle,total,1,P60021102
1124003,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,clinton,total,154,P00003392
1124004,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,fuente,total,0,P60016342
1124005,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,johnson,total,18,P60012234
1124006,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,stein,total,5,P20003984
1124007,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,trump,total,356,P80001571
1124008,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,in,total,4,NaN
1124009,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,mcmullin,total,7,P60022654
1124010,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,kennedy,total,0,NaN
1124011,2016,Minnesota,MN,27,33,Aitkin County,27001.0,663198.0,46.602446,-93.41976,Aitkin,AITKIN TWP|10,vacek,total,2,NaN


In [6]:
# Just look at precincts and votes for now, plus 'candidate_normalized' for groupby purposes
test2_df = test_df[['precinct', 'votes', 'candidate_normalized']]
test2_df

,precinct,votes,candidate_normalized
1124002,AITKIN TWP|10,1,castle
1124003,AITKIN TWP|10,154,clinton
1124004,AITKIN TWP|10,0,fuente
1124005,AITKIN TWP|10,18,johnson
1124006,AITKIN TWP|10,5,stein
1124007,AITKIN TWP|10,356,trump
1124008,AITKIN TWP|10,4,in
1124009,AITKIN TWP|10,7,mcmullin
1124010,AITKIN TWP|10,0,kennedy
1124011,AITKIN TWP|10,2,vacek


In [7]:
# Create a new column of 'total_votes' and first populate with 'Nan' values (is there a faster way to do this?)
test2_df['total_votes'] = 0
test2_df = test2_df[['precinct', 'votes', 'total_votes', 'candidate_normalized']]
test2_df

C:\Users\test\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,precinct,votes,total_votes,candidate_normalized
1124002,AITKIN TWP|10,1,0,castle
1124003,AITKIN TWP|10,154,0,clinton
1124004,AITKIN TWP|10,0,0,fuente
1124005,AITKIN TWP|10,18,0,johnson
1124006,AITKIN TWP|10,5,0,stein
1124007,AITKIN TWP|10,356,0,trump
1124008,AITKIN TWP|10,4,0,in
1124009,AITKIN TWP|10,7,0,mcmullin
1124010,AITKIN TWP|10,0,0,kennedy
1124011,AITKIN TWP|10,2,0,vacek


In [8]:
# TESTING GROUPBY JUST THIS CELL
# problem with groupby: still need to loop thru and sum the votes
test2_groupby = test2_df.groupby(['precinct', 'candidate_normalized']).sum()
test2_groupby

votes  total_votes
precinct      candidate_normalized                    
AITKIN TWP|10 castle                    1            0
              clinton                 154            0
              fuente                    0            0
              in                        4            0
              johnson                  18            0
              kennedy                   0            0
              mcmullin                  7            0
              stein                     5            0
              trump                   356            0
              vacek                     2            0
AITKIN|5      castle                    3            0
              clinton                 358            0
              fuente                    0            0
              in                        6            0
              johnson                  36            0
              kennedy                   1            0
              mcmullin                 21            0
              stein                     2            0
              trump                   552            0
              vacek                     7            0

In [9]:
# TESTING
print(test2_df.iloc[0, 1])
print("-----")
print("Want next line to print '154':")
print(test2_df.iloc[1, 1])
print("Printing Aitkin TWP|10?:")
print(test2_df.iloc[0,0])

1
-----
Want next line to print '154':
154
Printing Aitkin TWP|10?:
AITKIN TWP|10


In [28]:
vote_tallies = []
vote_total = 0

for i in range(0, len(test2_df)):
    # Precinct variables
    last_precinct = test2_df.iloc[i - 1, 0]
    precinct = test2_df.iloc[i, 0]
    
    # Vote variables
    vote = test2_df.iloc[i, 1]
    vote_total = vote_total + vote
    vote_tallies.append(vote)
    print(f"Vote: {vote} | Vote Total: {vote_total}")
    
    print(f"Precinct: {precinct}")
    print(f"Votes: {votes}")
    
    if precinct == last_precinct:
        print("same")
    else:
        print("changing")
    print("---------")
    
print("vote_tallies:")
print(vote_tallies)
print("vote_total: " + str(vote_total))

Vote: 1 | Vote Total: 1
Precinct: AITKIN TWP|10
Votes: 7
changing
---------
Vote: 154 | Vote Total: 155
Precinct: AITKIN TWP|10
Votes: 7
same
---------
Vote: 0 | Vote Total: 155
Precinct: AITKIN TWP|10
Votes: 7
same
---------
Vote: 18 | Vote Total: 173
Precinct: AITKIN TWP|10
Votes: 7
same
---------
Vote: 5 | Vote Total: 178
Precinct: AITKIN TWP|10
Votes: 7
same
---------
Vote: 356 | Vote Total: 534
Precinct: AITKIN TWP|10
Votes: 7
same
---------
Vote: 4 | Vote Total: 538
Precinct: AITKIN TWP|10
Votes: 7
same
---------
Vote: 7 | Vote Total: 545
Precinct: AITKIN TWP|10
Votes: 7
same
---------
Vote: 0 | Vote Total: 545
Precinct: AITKIN TWP|10
Votes: 7
same
---------
Vote: 2 | Vote Total: 547
Precinct: AITKIN TWP|10
Votes: 7
same
---------
Vote: 3 | Vote Total: 550
Precinct: AITKIN|5
Votes: 7
changing
---------
Vote: 358 | Vote Total: 908
Precinct: AITKIN|5
Votes: 7
same
---------
Vote: 0 | Vote Total: 908
Precinct: AITKIN|5
Votes: 7
same
---------
Vote: 36 | Vote Total: 944
Precinct: AIT

In [40]:
vote_tallies = []
vote_total = 0

for i in range(0, len(test2_df)):
    # Precinct variables
    last_precinct = test2_df.iloc[i - 1, 0]
    precinct = test2_df.iloc[i, 0]
    
    # Vote variables
    vote = test2_df.iloc[i, 1]
    vote_total = vote_total + vote
    vote_tallies.append(vote)
    print(f"Vote: {vote} | Vote Total: {vote_total}")
    
    print(f"Precinct: {precinct}")
        
    if precinct == last_precinct:
        print("same")
    else:
        print("changing")
    print("---------")
    
print("vote_tallies:")
print(vote_tallies)
print("vote_total: " + str(vote_total))

Vote: 1 | Vote Total: 987
Precinct: AITKIN TWP|10
changing
---------
Vote: 154 | Vote Total: 155
Precinct: AITKIN TWP|10
same
---------
Vote: 0 | Vote Total: 309
Precinct: AITKIN TWP|10
same
---------
Vote: 18 | Vote Total: 327
Precinct: AITKIN TWP|10
same
---------
Vote: 5 | Vote Total: 350
Precinct: AITKIN TWP|10
same
---------
Vote: 356 | Vote Total: 711
Precinct: AITKIN TWP|10
same
---------
Vote: 4 | Vote Total: 1071
Precinct: AITKIN TWP|10
same
---------
Vote: 7 | Vote Total: 1082
Precinct: AITKIN TWP|10
same
---------
Vote: 0 | Vote Total: 1089
Precinct: AITKIN TWP|10
same
---------
Vote: 2 | Vote Total: 1091
Precinct: AITKIN TWP|10
same
---------
Vote: 3 | Vote Total: 1096
Precinct: AITKIN|5
changing
---------
Vote: 358 | Vote Total: 361
Precinct: AITKIN|5
same
---------
Vote: 0 | Vote Total: 719
Precinct: AITKIN|5
same
---------
Vote: 36 | Vote Total: 755
Precinct: AITKIN|5
same
---------
Vote: 2 | Vote Total: 793
Precinct: AITKIN|5
same
---------
Vote: 552 | Vote Total: 1347


In [30]:
for i in range(10):
    test2_df.iloc[i, 2] = vote_total
test2_df

,precinct,votes,total_votes,candidate_normalized
1124002,AITKIN TWP|10,1,1533,castle
1124003,AITKIN TWP|10,154,1533,clinton
1124004,AITKIN TWP|10,0,1533,fuente
1124005,AITKIN TWP|10,18,1533,johnson
1124006,AITKIN TWP|10,5,1533,stein
1124007,AITKIN TWP|10,356,1533,trump
1124008,AITKIN TWP|10,4,1533,in
1124009,AITKIN TWP|10,7,1533,mcmullin
1124010,AITKIN TWP|10,0,1533,kennedy
1124011,AITKIN TWP|10,2,1533,vacek


In [ ]:
# Need to add a column with total precinct votes
# Will have one value for each precinct

In [ ]:
mn_df.groupby(['precinct']).sum()